In [2]:
%reload_ext autoreload
%autoreload 2

In [1]:
import json

from typing import Optional

import sqlite3
import instructor
import pandas as pd

from tqdm import tqdm
from openai import OpenAI
from pydantic import ValidationError

from elections import constants
from elections.utils import full_logger
from elections.data_schemas import ArticleSentiment
from elections.scrapers.news_scraper import NewsScraper
from elections.prompts.templates import sentiment_template


#logger = full_logger(constants.LOG_LVL, constants.SENTIMENT_LOG_FN, to_console=False)


class SentimentAnalysis:
    def __init__(self):
        self.client = instructor.patch(OpenAI())
        self.articles_df = pd.DataFrame()
        self.sentiments = []
        self.articles_counter = 0
        
    def load_articles(self, n_articles=None, refresh=False, query=None) -> None:
        if query is not None:
            self.articles_df = NewsScraper.load_articles(query)
            return
        if refresh:
            query = "SELECT article_id, title, description, text FROM articles"
        else:
            query = """
                SELECT 
                    atc.article_id, title, description, text
                FROM articles atc
                LEFT JOIN article_sentiments atc_s
                ON atc.article_id = atc_s.article_id
                WHERE atc_s.analysis IS NULL
            """
        if n_articles is not None:
            query = f"{query} LIMIT {n_articles}"
        self.articles_df = NewsScraper.load_articles(query)
    
    def get_article_sentiment(self, title, description, text) -> pd.DataFrame:
        article_n_meta = title + "\n" + description + "\n" + text
        politicians_present = ", ".join(
            [politician for politician in constants.POLITICIANS if politician in article_n_meta]
        )
        system_prompt = sentiment_template.SYSTEM_PROMPT.format(politicians=politicians_present)
        user_prompt = sentiment_template.USER_PROMPT.format(title=title, description=description, text=text)
        
        try:
            # to see the raw response: resp._raw_response.model_dump_json(indent=2)
            resp = self.client.chat.completions.create(
                model=constants.OPENAI_GPT_MODEL,
                response_model=ArticleSentiment,
                validation_context={
                    "article_n_meta": article_n_meta,
                    "politicians_present": politicians_present
                },
                max_retries=constants.MAX_RETRIES,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt},
                ],
            )
        except ValidationError:
            resp = None
        
        return pd.DataFrame({"analysis": [resp], "system_prompt": [system_prompt], "user_prompt": [user_prompt]})
        
    
    def get_sentiments(self, freq=5, save=True) -> Optional[pd.DataFrame]:
        assert not self.articles_df.empty, "No articles loaded"
        N = len(self.articles_df)
        
        if save:
            engine = sqlite3.connect(constants.NEWS_DB)
        
        self.sentiments = []
        for i, row in tqdm(self.articles_df.reset_index().iterrows(), total=N):
            #if i % freq == 0:
                #logger.info(f"Processing article {i + 1} of {N}")
                #print(f"Processing article {i + 1} of {N}")
            sentiment = self.get_article_sentiment(row["title"], row["description"], row["text"])
            sentiment.insert(loc=0, column="article_id", value=row["article_id"])
            if not sentiment.empty:
                self.sentiments.append(sentiment)
                self.articles_counter += 1
                if self.articles_counter % freq == 0 or i == N - 1:
                    if save:
                        self._save_sentiments(engine)
                    else:
                        #logger.info(f"Extracted {self.articles_counter} of {N}")
                        print(f"Extracted {self.articles_counter} of {N}")
        if save:
            engine.close()
            return None
        
        return self.sentiments
    
    def _save_sentiments(self, engine) -> None:
        sentiments_df = pd.concat(self.sentiments)
        sentiments_df["analysis"] = sentiments_df["analysis"].apply(lambda x: x.model_dump_json())
        sentiments_df.to_sql("article_sentiments", con=engine, if_exists="append", index=False)
        self.sentiments = []
        #logger.info(f"Saved in DB {self.articles_counter} of {N}")
        print(f"Saved in DB {self.articles_counter} of {len(self.articles_df)}")
    
    def load_article_sentiments(query) -> pd.DataFrame:
        with sqlite3.connect(constants.NEWS_DB) as engine:
            df = pd.read_sql("SELECT * FROM article_sentiments", con=engine)
        if "analysis" in df.columns:
            df["analysis"] = df["analysis"].apply(lambda x: ArticleSentiment.model_validate_json(x))
        return df
        

        

In [3]:
sentiment_analysis = SentimentAnalysis()
sentiment_analysis.load_articles() #n_articles=2)#, refresh=True)
#sentiment_analysis.articles_df
sentiment_analysis.get_sentiments(freq=1, save=False)

  0%|          | 1/684 [00:04<45:50,  4.03s/it]

Saved in DB 1 of 684


03/04/2024 04:31:50 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
  0%|          | 2/684 [00:12<1:16:23,  6.72s/it]

Saved in DB 2 of 684


03/04/2024 04:31:52 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
  0%|          | 3/684 [00:14<53:08,  4.68s/it]  

...

Saved in DB 17 of 684


03/04/2024 04:33:11 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
03/04/2024 04:33:35 PM - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
  3%|▎         | 18/684 [01:58<2:25:44, 13.13s/it]

Saved in DB 18 of 684


  3%|▎         | 18/684 [01:58<1:12:56,  6.57s/it]


KeyboardInterrupt: 